In [1]:
from brevitas.core.scaling import ConstScaling
from brevitas.core.quant.int_base import IntQuant
import torch
import torch.nn as nn
from torch.nn.functional import linear

In [5]:
int_quant = IntQuant(narrow_range=True, signed=False)
print(torch.tensor(4.))
scale, zero_point, bit_width = torch.tensor(0.01), torch.tensor(0.), torch.tensor(2.)
inp = torch.Tensor([0.042, -0.053, 0.31, -0.44])
out = int_quant(scale, zero_point, bit_width, inp)
print(out)

tensor(4.)
tensor([0.0200, 0.0000, 0.0200, 0.0000])


In [9]:
torch.Tensor(5,5)

tensor([[ 4.1672e+11,  4.5705e-41, -4.9658e+03,  3.0866e-41, -2.1834e-11],
        [ 4.5703e-41, -2.1834e-11,  4.5703e-41, -2.1834e-11,  4.5703e-41],
        [-2.1833e-11,  4.5703e-41, -2.1832e-11,  4.5703e-41, -2.1834e-11],
        [ 4.5703e-41, -2.1832e-11,  4.5703e-41, -2.1834e-11,  4.5703e-41],
        [-2.1832e-11,  4.5703e-41, -2.1834e-11,  4.5703e-41, -2.1832e-11]])

Creación de la linear custom layer

In [19]:
class LinearC(nn.Module):
    def __init__(self,size_in,size_out):
        super(LinearC,self).__init__()
        self.size_in = size_in
        self.size_out = size_out
        weight = torch.Tensor(self.size_out,self.size_in)
        self.weight = nn.Parameter(weight)
        nn.init.xavier_normal_(self.weight,1)

    def forward(self,x):
        output = linear(x,self.weight,bias=None)
        return torch.round(input=output,decimals=1)


In [7]:
capa1 = LinearC(5,2)
capa2 = LinearC(2,1)

In [14]:
tensor = torch.ones(10,5)
capa1(tensor)

tensor([[-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267],
        [-0.9977, -1.2267]], grad_fn=<MmBackward0>)

In [19]:
net = nn.Sequential(
    capa1,
    capa2
)

In [20]:
net(tensor)

tensor([[-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000],
        [-1.6000]], grad_fn=<RoundBackward1>)

In [21]:
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            LinearC(5,4),
            nn.ReLU(),
            LinearC(4,10)
        )

    def forward(self,x):
        return self.linear_relu_stack(x)
        
    

In [22]:
net = CustomNet()


In [23]:
net(tensor)

tensor([[-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -0.6000,  0.4000, -0.3000, -1.0000,  1.1000, -0.7000,  0.5000,
         -1.4000,  1.1000],
        [-1.8000, -